In [69]:
import os
import pandas as pd
import numpy as np
from matplotlib import pylab as plt

In [2]:
token = os.environ['oanda_access_token']
account = os.environ['oanda_account_id']

In [32]:
import optuna
from oanda_bot import Bot

In [101]:
class MyBot(Bot):
    def set_param(self, a, b, stop_loss, take_profit):
        self.a = a
        self.b = b
        self.stop_loss = stop_loss
        self.take_profit = take_profit
    
    def strategy(self):
        fast_ma = self.sma(period=self.a)
        slow_ma = self.sma(period=self.b)
        # golden cross
        self.sell_exit = self.buy_entry = (fast_ma > slow_ma) & (
            fast_ma.shift() <= slow_ma.shift()
        )
        # dead cross
        self.buy_exit = self.sell_entry = (fast_ma < slow_ma) & (
            fast_ma.shift() >= slow_ma.shift()
        )

    def backtest(
            self, *, from_date: str = "", to_date: str = "", filename: str = ""
        ) -> None:
            csv = "{}-{}-{}-{}.csv".format(
                self.instrument, self.granularity, from_date, to_date
            )
            if os.path.exists(csv):
                self.df = pd.read_csv(
                    csv, index_col=0, parse_dates=True, infer_datetime_format=True
                )
            else:
                self._candles(from_date=from_date, to_date=to_date)
                if from_date != "" and to_date != "":
                    self.df.to_csv(csv)
            self.strategy()
            o = self.df.O.values
            L = self.df.L.values
            h = self.df.H.values
            N = len(self.df)
            long_trade = np.zeros(N)
            short_trade = np.zeros(N)

            # buy entry
            buy_entry_s = np.hstack((False, self.buy_entry[:-1]))  # shift
            long_trade[buy_entry_s] = o[buy_entry_s]
            # buy exit
            buy_exit_s = np.hstack((False, self.buy_exit[:-2], True))  # shift
            long_trade[buy_exit_s] = -o[buy_exit_s]
            # sell entry
            sell_entry_s = np.hstack((False, self.sell_entry[:-1]))  # shift
            short_trade[sell_entry_s] = o[sell_entry_s]
            # sell exit
            sell_exit_s = np.hstack((False, self.sell_exit[:-2], True))  # shift
            short_trade[sell_exit_s] = -o[sell_exit_s]

            long_pl = pd.Series(np.zeros(N))  # profit/loss of buy position
            short_pl = pd.Series(np.zeros(N))  # profit/loss of sell position
            buy_price = sell_price = 0
            long_rr = []  # long return rate
            short_rr = []  # short return rate
            stop_loss = take_profit = 0

            for i in range(1, N):
                # buy entry
                if long_trade[i] > 0:
                    if buy_price == 0:
                        buy_price = long_trade[i]
                        short_trade[i] = -buy_price  # sell exit
                    else:
                        long_trade[i] = 0

                # sell entry
                if short_trade[i] > 0:
                    if sell_price == 0:
                        sell_price = short_trade[i]
                        long_trade[i] = -sell_price  # buy exit
                    else:
                        short_trade[i] = 0

                # buy exit
                if long_trade[i] < 0:
                    if buy_price != 0:
                        long_pl[i] = (
                            -(buy_price + long_trade[i]) * self.units
                        )  # profit/loss fixed
                        long_rr.append(
                            round(long_pl[i] / buy_price * 100, 2)
                        )  # long return rate
                        buy_price = 0
                    else:
                        long_trade[i] = 0

                # sell exit
                if short_trade[i] < 0:
                    if sell_price != 0:
                        short_pl[i] = (
                            sell_price + short_trade[i]
                        ) * self.units  # profit/loss fixed
                        short_rr.append(
                            round(short_pl[i] / sell_price * 100, 2)
                        )  # short return rate
                        sell_price = 0
                    else:
                        short_trade[i] = 0

                # close buy position with stop loss
                if buy_price != 0 and self.stop_loss > 0:
                    stop_price = buy_price - self.stop_loss * self.point
                    if L[i] <= stop_price:
                        long_trade[i] = -stop_price
                        long_pl[i] = (
                            -(buy_price + long_trade[i]) * self.units
                        )  # profit/loss fixed
                        long_rr.append(
                            round(long_pl[i] / buy_price * 100, 2)
                        )  # long return rate
                        buy_price = 0
                        stop_loss += 1

                # close buy positon with take profit
                if buy_price != 0 and self.take_profit > 0:
                    limit_price = buy_price + self.take_profit * self.point
                    if h[i] >= limit_price:
                        long_trade[i] = -limit_price
                        long_pl[i] = (
                            -(buy_price + long_trade[i]) * self.units
                        )  # profit/loss fixed
                        long_rr.append(
                            round(long_pl[i] / buy_price * 100, 2)
                        )  # long return rate
                        buy_price = 0
                        take_profit += 1

                # close sell position with stop loss
                if sell_price != 0 and self.stop_loss > 0:
                    stop_price = sell_price + self.stop_loss * self.point
                    if h[i] >= stop_price:
                        short_trade[i] = -stop_price
                        short_pl[i] = (
                            sell_price + short_trade[i]
                        ) * self.units  # profit/loss fixed
                        short_rr.append(
                            round(short_pl[i] / sell_price * 100, 2)
                        )  # short return rate
                        sell_price = 0
                        stop_loss += 1

                # close sell position with take profit
                if sell_price != 0 and self.take_profit > 0:
                    limit_price = sell_price - self.take_profit * self.point
                    if L[i] <= limit_price:
                        short_trade[i] = -limit_price
                        short_pl[i] = (
                            sell_price + short_trade[i]
                        ) * self.units  # profit/loss fixed
                        short_rr.append(
                            round(short_pl[i] / sell_price * 100, 2)
                        )  # short return rate
                        sell_price = 0
                        take_profit += 1

            win_trades = np.count_nonzero(long_pl.clip(lower=0)) + np.count_nonzero(
                short_pl.clip(lower=0)
            )
            lose_trades = np.count_nonzero(long_pl.clip(upper=0)) + np.count_nonzero(
                short_pl.clip(upper=0)
            )
            trades = (np.count_nonzero(long_trade) // 2) + (
                np.count_nonzero(short_trade) // 2
            )
            gross_profit = long_pl.clip(lower=0).sum() + short_pl.clip(lower=0).sum()
            gross_loss = long_pl.clip(upper=0).sum() + short_pl.clip(upper=0).sum()
            profit_pl = gross_profit + gross_loss
            self.equity = (long_pl + short_pl).cumsum()
            mdd = (self.equity.cummax() - self.equity).max()
            self.return_rate = pd.Series(short_rr + long_rr)

            s = pd.Series(dtype="object")
            s.loc["total profit"] = round(profit_pl, 3)
            s.loc["total trades"] = trades
            s.loc["win rate"] = round(win_trades / trades * 100, 3)
            s.loc["profit factor"] = round(-gross_profit / gross_loss, 3)
            s.loc["maximum drawdown"] = round(mdd, 3)
            s.loc["recovery factor"] = round(profit_pl / mdd, 3)
            s.loc["riskreward ratio"] = round(
                -(gross_profit / win_trades) / (gross_loss / lose_trades), 3
            )
            s.loc["sharpe ratio"] = round(
                self.return_rate.mean() / self.return_rate.std(), 3
            )
            s.loc["average return"] = round(self.return_rate.mean(), 3)
            s.loc["stop loss"] = stop_loss
            s.loc["take profit"] = take_profit
#             print(s)

#             fig = plt.figure()
#             fig.subplots_adjust(
#                 wspace=0.2, hspace=0.5, left=0.095, right=0.95, bottom=0.095, top=0.95
#             )
#             ax1 = fig.add_subplot(3, 1, 1)
#             ax1.plot(self.df.C, label="close")
#             ax1.legend()
#             ax2 = fig.add_subplot(3, 1, 2)
#             ax2.plot(self.equity, label="equity")
#             ax2.legend()
#             ax3 = fig.add_subplot(3, 1, 3)
#             ax3.hist(self.return_rate, 50, rwidth=0.9)
#             ax3.axvline(
#                 sum(self.return_rate) / len(self.return_rate),
#                 color="orange",
#                 label="average return",
#             )
#             ax3.legend()
#             if filename == "":
#                 plt.show()
#             else:
#                 plt.savefig(filename)

            return s

In [102]:
mybot = MyBot(
    account_id=account,
    access_token=token,
    environment='live',
    instrument='USD_JPY',
    granularity='H1', # 15 second candlestick,
)

In [103]:
def objective(trial):
    a = trial.suggest_int('a', 5, 15)
    b = trial.suggest_int('b', 16, 30)
    sl = trial.suggest_int('stop_loss', 10, 50)
    tp = trial.suggest_int('take_profit', 10, 50)
    
    mybot.set_param(a, b, sl, tp)
    
    score =  -mybot.backtest(from_date="2020-12-1", to_date="2021-2-14", filename="backtest.png")['average return']
    print('a: %d, b: %d, score: %1.2f' % (a, b, -score))
    return score

In [104]:
study = optuna.create_study()
study.optimize(objective, n_trials=1000)

[I 2021-02-16 22:22:51,562] A new study created in memory with name: no-name-6e562153-eda9-4a07-b1d4-4074bd237a1b
[I 2021-02-16 22:22:51,642] Trial 0 finished with value: -24.398 and parameters: {'a': 15, 'b': 23, 'stop_loss': 34, 'take_profit': 38}. Best is trial 0 with value: -24.398.
[I 2021-02-16 22:22:51,708] Trial 1 finished with value: -181.363 and parameters: {'a': 10, 'b': 19, 'stop_loss': 16, 'take_profit': 49}. Best is trial 1 with value: -181.363.
[I 2021-02-16 22:22:51,762] Trial 2 finished with value: -37.605 and parameters: {'a': 14, 'b': 29, 'stop_loss': 16, 'take_profit': 20}. Best is trial 1 with value: -181.363.
[I 2021-02-16 22:22:51,830] Trial 3 finished with value: -258.666 and parameters: {'a': 13, 'b': 25, 'stop_loss': 50, 'take_profit': 44}. Best is trial 3 with value: -258.666.


a: 15, b: 23, score: 24.40
a: 10, b: 19, score: 181.36
a: 14, b: 29, score: 37.60
a: 13, b: 25, score: 258.67


[I 2021-02-16 22:22:51,892] Trial 4 finished with value: -244.963 and parameters: {'a': 11, 'b': 24, 'stop_loss': 29, 'take_profit': 29}. Best is trial 3 with value: -258.666.
[I 2021-02-16 22:22:51,952] Trial 5 finished with value: 124.883 and parameters: {'a': 11, 'b': 16, 'stop_loss': 38, 'take_profit': 10}. Best is trial 3 with value: -258.666.
[I 2021-02-16 22:22:52,014] Trial 6 finished with value: -342.14 and parameters: {'a': 9, 'b': 29, 'stop_loss': 38, 'take_profit': 43}. Best is trial 6 with value: -342.14.
[I 2021-02-16 22:22:52,080] Trial 7 finished with value: -52.691 and parameters: {'a': 15, 'b': 16, 'stop_loss': 14, 'take_profit': 30}. Best is trial 6 with value: -342.14.


a: 11, b: 24, score: 244.96
a: 11, b: 16, score: -124.88
a: 9, b: 29, score: 342.14
a: 15, b: 16, score: 52.69


[I 2021-02-16 22:22:52,142] Trial 8 finished with value: -274.86 and parameters: {'a': 10, 'b': 25, 'stop_loss': 36, 'take_profit': 25}. Best is trial 6 with value: -342.14.
[I 2021-02-16 22:22:52,199] Trial 9 finished with value: -135.594 and parameters: {'a': 14, 'b': 23, 'stop_loss': 49, 'take_profit': 27}. Best is trial 6 with value: -342.14.
[I 2021-02-16 22:22:52,288] Trial 10 finished with value: -94.723 and parameters: {'a': 5, 'b': 30, 'stop_loss': 42, 'take_profit': 39}. Best is trial 6 with value: -342.14.


...

a: 9, b: 27, score: 564.97
a: 9, b: 27, score: 583.52


[I 2021-02-16 22:24:35,107] Trial 997 finished with value: -268.273 and parameters: {'a': 8, 'b': 26, 'stop_loss': 19, 'take_profit': 44}. Best is trial 992 with value: -734.625.
[I 2021-02-16 22:24:35,222] Trial 998 finished with value: -644.499 and parameters: {'a': 9, 'b': 28, 'stop_loss': 13, 'take_profit': 45}. Best is trial 992 with value: -734.625.


a: 8, b: 26, score: 268.27
a: 9, b: 28, score: 644.50


[I 2021-02-16 22:24:35,341] Trial 999 finished with value: -765.499 and parameters: {'a': 9, 'b': 29, 'stop_loss': 12, 'take_profit': 44}. Best is trial 999 with value: -765.499.


a: 9, b: 29, score: 765.50
